# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [11]:
# your code here
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('tic-tac-toe.csv')
df.head()
df.replace({'x': 1, 'o': 0, 'b': -1, True: 1, False: 0}, inplace=True)
X = df.drop('class', axis=1)
y = df['class']

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [12]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = Sequential()

model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax')) 

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

model.save('tic-tac-toe.model')


Epoch 1/10
20/20 [==============================] - 3s 47ms/step - loss: 0.6376 - accuracy: 0.6389 - val_loss: 0.6096 - val_accuracy: 0.6818
Epoch 2/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5995 - accuracy: 0.6699 - val_loss: 0.6178 - val_accuracy: 0.6818
Epoch 3/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5752 - accuracy: 0.7092 - val_loss: 0.6068 - val_accuracy: 0.7013
Epoch 4/10
20/20 [==============================] - 0s 6ms/step - loss: 0.5550 - accuracy: 0.7239 - val_loss: 0.6009 - val_accuracy: 0.7143
Epoch 5/10
20/20 [==============================] - 0s 11ms/step - loss: 0.5339 - accuracy: 0.7565 - val_loss: 0.5847 - val_accuracy: 0.6948
Epoch 6/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5127 - accuracy: 0.7647 - val_loss: 0.5747 - val_accuracy: 0.7143
Epoch 7/10
20/20 [==============================] - 0s 9ms/step - loss: 0.4961 - accuracy: 0.7778 - val_loss: 0.5577 - val_accuracy: 0.7468
Epoch 8/10
20/20 [

INFO:tensorflow:Assets written to: tic-tac-toe.model/assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [13]:
from tensorflow.keras.models import load_model

model = load_model('tic-tac-toe.model')

In [15]:
# your code here
import numpy as np


num_samples = 5
random_indices = np.random.choice(X_test.shape[0], num_samples, replace=False)
X_sample = X_test[random_indices]
y_sample_actual = y_test.iloc[random_indices]
y_sample_pred = model.predict(X_sample)
y_sample_pred_classes = np.argmax(y_sample_pred, axis=1)
for i in range(num_samples):
    print(f"Sample {i+1}:")
    print(f"Predicted: {y_sample_pred_classes[i]}, Actual: {y_sample_actual.iloc[i]}")


1/1 [==============================] - 0s 123ms/step
Sample 1:
Predicted: 1, Actual: 0
Sample 2:
Predicted: 1, Actual: 0
Sample 3:
Predicted: 1, Actual: 1
Sample 4:
Predicted: 1, Actual: 1
Sample 5:
Predicted: 1, Actual: 1


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [22]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))  

from tensorflow.keras.optimizers import Adam

learning_rate = 0.001  
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=30, validation_split=0.2)



Epoch 1/30
20/20 [==============================] - 2s 23ms/step - loss: 0.6395 - accuracy: 0.6454 - val_loss: 0.6080 - val_accuracy: 0.6818
Epoch 2/30
20/20 [==============================] - 0s 10ms/step - loss: 0.5831 - accuracy: 0.6667 - val_loss: 0.5998 - val_accuracy: 0.7078
Epoch 3/30
20/20 [==============================] - 0s 7ms/step - loss: 0.5447 - accuracy: 0.7222 - val_loss: 0.5852 - val_accuracy: 0.6818
Epoch 4/30
20/20 [==============================] - 0s 12ms/step - loss: 0.5005 - accuracy: 0.7794 - val_loss: 0.5639 - val_accuracy: 0.7208
Epoch 5/30
20/20 [==============================] - 0s 11ms/step - loss: 0.4664 - accuracy: 0.8170 - val_loss: 0.5239 - val_accuracy: 0.7727
Epoch 6/30
20/20 [==============================] - 0s 6ms/step - loss: 0.4018 - accuracy: 0.8448 - val_loss: 0.4832 - val_accuracy: 0.7792
Epoch 7/30
20/20 [==============================] - 0s 6ms/step - loss: 0.3576 - accuracy: 0.8595 - val_loss: 0.4568 - val_accuracy: 0.8052
Epoch 8/30
20/20

In [23]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

6/6 [==============================] - 0s 3ms/step - loss: 0.2861 - accuracy: 0.8906
Test accuracy: 0.890625


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
#adjusting learing rate and adding more layers